<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_MP2_2_SkimLit_Full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Milestone Project 2.2: SkimLit Full

Esta é a continuidade do projeto SkimLit+.

Neste projeto a base de dados PubMed 200k RCT completa é utilizada.

* `pubmed-rct/PubMed_200k_RCT/train.txt`
* `pubmed-rct/PubMed_200k_RCT/test.txt`

# Importações

In [ ]:
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.data import AUTOTUNE, Dataset
from tensorflow.keras import Model
from tensorflow.keras.layers import Bidirectional, Concatenate, Dense, Dropout, Embedding, Input, LSTM, TextVectorization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Configurações e constantes

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100

plt.style.use('seaborn-darkgrid')

COR = '#007f66'

DIRETORIO = '/content/pubmed-rct/PubMed_200k_RCT'

LIMITE_DICIONARIO = 331000
LIMITE_CARACTERES = 80

LIMITE_INCORPORADOR = 300
LIMITE_INCORPORADOR_CARACTERES = 25
LIMITE_VETOR_TEXTO = 100

LOTE_TAMANHO = 32

ENTRADA_FORMATO = (1,)
ENTRADA_TIPO = tf.string

ATIVACAO_CNN = 'relu'
ATIVACAO_RNN = 'tanh'
ATIVACAO_SAIDA = 'softmax'

SUAVIZACAO = 0.2
APRENDIZADO = 0.001
METRICAS = ['accuracy']

ITERACOES = 3

# Funções

In [ ]:
!wget https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py

In [ ]:
from funcoes import avaliar_modelo, grafico_historico_por_iteracao, preprocessar_texto

# Dados

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git

!7z e -o$DIRETORIO $DIRETORIO/train.7z

In [ ]:
df_treino = pd.DataFrame(preprocessar_texto(f'{DIRETORIO}/train.txt'))
df_teste  = pd.DataFrame(preprocessar_texto(f'{DIRETORIO}/test.txt'))

codificador_onehot = OneHotEncoder(sparse=False)

rotulos_onehot_treino = codificador_onehot.fit_transform(df_treino['classe'].to_numpy().reshape(-1, 1))
rotulos_onehot_teste  = codificador_onehot.transform(df_teste['classe'].to_numpy().reshape(-1, 1))

codificador_int = LabelEncoder()

rotulos_int_treino = codificador_int.fit_transform(df_treino['classe'].to_numpy())
rotulos_int_teste  = codificador_int.transform(df_teste['classe'].to_numpy())

classes = codificador_int.classes_

palavras_por_texto_98   = int(np.percentile([len(texto.split()) for texto in df_treino['texto']], 98))
caracteres_por_texto_98 = int(np.percentile([len(texto) for texto in df_treino['texto']], 98))

lista_caracteres_treino = [" ".join(list(texto)) for texto in df_treino['texto']]
lista_caracteres_teste  = [" ".join(list(texto)) for texto in df_teste['texto']]

numero_linha_98 = int(np.percentile(df_treino['numero'], 98))

linhas_numero_onehot_treino = tf.one_hot(df_treino['numero'], depth=numero_linha_98)
linhas_numero_onehot_teste  = tf.one_hot(df_teste['numero'], depth=numero_linha_98)

total_linhas_98 = int(np.percentile(df_treino['total'], 98))

linhas_total_onehot_treino = tf.one_hot(df_treino['total'], depth=total_linhas_98)
linhas_total_onehot_teste  = tf.one_hot(df_teste['total'], depth=total_linhas_98)

dados_hibridos_linhas_treino_textos  = Dataset.from_tensor_slices((df_treino['texto'], lista_caracteres_treino, linhas_numero_onehot_treino, linhas_total_onehot_treino))
dados_hibridos_linhas_treino_rotulos = Dataset.from_tensor_slices(rotulos_onehot_treino)
dados_hibridos_linhas_treino         = Dataset.zip((dados_hibridos_linhas_treino_textos, dados_hibridos_linhas_treino_rotulos))
dados_hibridos_linhas_treino         = dados_hibridos_linhas_treino.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)

dados_hibridos_linhas_teste_textos  = Dataset.from_tensor_slices((df_teste['texto'], lista_caracteres_teste, linhas_numero_onehot_teste, linhas_total_onehot_teste))
dados_hibridos_linhas_teste_rotulos = Dataset.from_tensor_slices(rotulos_onehot_teste)
dados_hibridos_linhas_teste         = Dataset.zip((dados_hibridos_linhas_teste_textos, dados_hibridos_linhas_teste_rotulos))
dados_hibridos_linhas_teste         = dados_hibridos_linhas_teste.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)

# Camadas auxiliares

In [ ]:
vetorizador_palavras = TextVectorization(max_tokens=LIMITE_DICIONARIO,
                                         output_mode='int',
                                         output_sequence_length=palavras_por_texto_98,
                                         name='vetorizador_palavras')

vetorizador_palavras.adapt(df_treino['texto'])

vocabulario_palavras = vetorizador_palavras.get_vocabulary()

incorporador_palavras = Embedding(input_dim=len(vocabulario_palavras),
                                  output_dim=LIMITE_INCORPORADOR,
                                  mask_zero=True,
                                  input_length=palavras_por_texto_98,
                                  name='incorporador_palavras')

vetorizador_caracteres = TextVectorization(max_tokens=LIMITE_CARACTERES,
                                           output_mode='int',
                                           output_sequence_length=caracteres_por_texto_98,
                                           name='vetorizador_caracteres')

vetorizador_caracteres.adapt(lista_caracteres_treino)

vocabulario_caracteres = vetorizador_caracteres.get_vocabulary()

incorporador_caracteres = Embedding(input_dim=len(vocabulario_caracteres),
                                    output_dim=LIMITE_INCORPORADOR_CARACTERES,
                                    mask_zero=True,
                                    input_length=caracteres_por_texto_98,
                                    name='incorporador_caracteres')

# Modelo

In [ ]:
modelo_nome = 'modelo_incorporacao_palavras'


entradas = Input(shape=ENTRADA_FORMATO, dtype=ENTRADA_TIPO, name='camada_entrada_palavras')

camadas = vetorizador_palavras(entradas)
camadas = incorporador_palavras(camadas)

saidas = Bidirectional(layer=LSTM(units=int(LIMITE_INCORPORADOR / 2), activation=ATIVACAO_RNN), name='camada_bi_lstm_palavras')(camadas)

modelo_incorporacao_palavras = Model(inputs=entradas, outputs=saidas, name=modelo_nome)


modelo_nome = 'modelo_incorporacao_caracteres'

entradas = Input(shape=ENTRADA_FORMATO, dtype=ENTRADA_TIPO, name='camada_entrada_caracteres')

camadas = vetorizador_caracteres(entradas)
camadas = incorporador_caracteres(camadas)

saidas = Bidirectional(layer=LSTM(units=LIMITE_INCORPORADOR_CARACTERES, activation=ATIVACAO_RNN), name='camada_bi_lstm_caracteres')(camadas)

modelo_incorporacao_caracteres = Model(inputs=entradas, outputs=saidas, name=modelo_nome)


combinacao_incorporacao = Concatenate(name="camada_combinacao_incorporacao")([modelo_incorporacao_palavras.output,
                                                                              modelo_incorporacao_caracteres.output])

abandonos_incorporacao = Dense(LIMITE_VETOR_TEXTO * 2, activation=ATIVACAO_CNN, name='camada_abandonos_incorporacao_relu')(combinacao_incorporacao)
abandonos_incorporacao = Dropout(rate=0.5, name='camada_abandonos_incorporacao')(abandonos_incorporacao)


modelo_nome = 'modelo_numero_linhas'

entradas = Input(shape=(numero_linha_98,), dtype=linhas_numero_onehot_treino.dtype, name='camada_entrada_numero_linhas')

saidas = Dense(numero_linha_98 * 2, activation=ATIVACAO_CNN, name='camada_relu_numero_linhas')(entradas)

modelo_numero_linhas = Model(inputs=entradas, outputs=saidas, name=modelo_nome)


modelo_nome = 'modelo_total_linhas'

entradas = Input(shape=(total_linhas_98,), dtype=linhas_total_onehot_treino.dtype, name='camada_entrada_total_linhas')

saidas = Dense(total_linhas_98 * 2, activation=ATIVACAO_CNN, name='camada_relu_total_linhas')(entradas)

modelo_total_linhas = Model(inputs=entradas, outputs=saidas, name=modelo_nome)


combinacao_linhas = Concatenate(name="camada_combinacao_linhas")([abandonos_incorporacao,
                                                                  modelo_numero_linhas.output,
                                                                  modelo_total_linhas.output])

abandonos_linhas = Dense(LIMITE_VETOR_TEXTO * 2, activation=ATIVACAO_CNN, name='camada_abandonos_linhas_relu')(combinacao_linhas)
abandonos_linhas = Dropout(rate=0.5, name='camada_abandonos_linhas')(abandonos_linhas)

saidas = Dense(len(classes), activation=ATIVACAO_SAIDA, name='camada_saida')(abandonos_linhas)


modelo_nome = 'modelo_pubmed_200k_rct'

modelo = Model(inputs=[modelo_incorporacao_palavras.input,
                       modelo_incorporacao_caracteres.input,
                       modelo_numero_linhas.input,
                       modelo_total_linhas.input],
               outputs=saidas,
               name=modelo_nome)

modelo.compile(loss=CategoricalCrossentropy(label_smoothing=SUAVIZACAO),
               optimizer=SGD(learning_rate=APRENDIZADO),
               metrics=METRICAS)

## Verificação do modelo

In [ ]:
modelo.summary()

In [ ]:
plot_model(modelo, show_shapes=True)

## Finalização do modelo

In [ ]:
historico = modelo.fit(dados_hibridos_linhas_treino,
                       epochs=ITERACOES,
                       validation_data=dados_hibridos_linhas_teste,
                       verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico)

In [ ]:
probabilidades = modelo.predict(dados_hibridos_linhas_teste)
previsoes      = tf.argmax(probabilidades, axis=1)
avaliacoes     = pd.DataFrame(avaliar_modelo(rotulos_int_teste, previsoes, classes))

In [ ]:
avaliacoes